In [1]:
import pandas as pd
from ide_utils.input_tables import load_dask
from ide_utils.output_table import send_to_output_table

data = load_dask("data").compute().drop('__index_level_0__', axis=1)
data.head()

,Fecha,Linea,Horas,Qty_passangers,uploaded,festivos,window_daily,Mediana,promedio,relacion,dia_semana,window_weekly,Mediana_semanal,promedio_semanal,relacion_semanal
0,2021-01-08,1,05,4915.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 614.0}, {'item': 1291.0}, {...",4609.5,3464.055556,1.330666,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5390.0,4681.571429,1.151323
1,2021-01-15,1,05,5201.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 4658.0}, {'item': 4745.0}, ...",4915.0,4155.190476,1.182858,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5424.0,4806.000000,1.128589
2,2021-01-22,1,05,5390.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 5248.0}, {'item': 5189.0}, ...",5311.0,4562.523810,1.164049,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5458.0,4934.222222,1.106152
3,2021-01-29,1,05,5458.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 5311.0}, {'item': 5329.0}, ...",5444.0,4702.238095,1.157747,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5514.5,5048.600000,1.092283
4,2021-02-05,1,05,5571.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 5462.0}, {'item': 5547.0}, ...",5547.0,4798.238095,1.156049,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5571.0,5149.454545,1.081862


In [5]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy import stats
from statsmodels.stats.multicomp import MultiComparison

In [4]:
# Cargando el dataframe
df = data

# Mostrando las primeras filas para entender la estructura y los tipos de datos
df.head()

,Fecha,Linea,Horas,Qty_passangers,uploaded,festivos,window_daily,Mediana,promedio,relacion,dia_semana,window_weekly,Mediana_semanal,promedio_semanal,relacion_semanal
0,2021-01-08,1,05,4915.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 614.0}, {'item': 1291.0}, {...",4609.5,3464.055556,1.330666,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5390.0,4681.571429,1.151323
1,2021-01-15,1,05,5201.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 4658.0}, {'item': 4745.0}, ...",4915.0,4155.190476,1.182858,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5424.0,4806.000000,1.128589
2,2021-01-22,1,05,5390.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 5248.0}, {'item': 5189.0}, ...",5311.0,4562.523810,1.164049,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5458.0,4934.222222,1.106152
3,2021-01-29,1,05,5458.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 5311.0}, {'item': 5329.0}, ...",5444.0,4702.238095,1.157747,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5514.5,5048.600000,1.092283
4,2021-02-05,1,05,5571.0,2023-11-16 13:20:23.698531,0,"{'list': [{'item': 5462.0}, {'item': 5547.0}, ...",5547.0,4798.238095,1.156049,Friday,"{'list': [{'item': 614.0}, {'item': 4915.0}, {...",5571.0,5149.454545,1.081862


## Test de anova para detectar diferencias significativas entre los días de la semana para cada combinación única de línea y hora


In [5]:
import pandas as pd
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

def realizar_anova(archivo_csv):
    """
    Realiza un análisis de varianza (ANOVA) para determinar si hay diferencias significativas
    en la cantidad de pasajeros entre los días de la semana para cada combinación única de línea y hora.

    Parámetros:
    - archivo_csv (str): Ruta al archivo CSV que contiene los datos.

    Retorna:
    - df_resultados (pd.DataFrame): DataFrame que contiene los resultados de las pruebas de ANOVA.

    Explicación:
    Un valor p menor a 0.05 indica que hay evidencia suficiente para rechazar la hipótesis nula.
    En este contexto, la hipótesis nula (H0) asume que las medias de la cantidad de pasajeros son iguales
    para todos los días de la semana en una línea y hora específicas. La hipótesis alternativa (H1)
    sugiere que al menos una de las medias es diferente.

    Por lo tanto:
    - Si Valor_p < 0.05: Se rechaza H0, lo que sugiere que hay diferencias significativas en la cantidad
      de pasajeros entre los días de la semana para esa combinación específica de línea y hora.
    - Si Valor_p >= 0.05: No hay evidencia suficiente para rechazar H0, lo que indica que no hay
      diferencias significativas en la cantidad de pasajeros entre los días de la semana.

    """
    
    # Manipulación de fechas
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['dia_semana'] = df['Fecha'].dt.day_name()

    # Inicialización de la lista de resultados de ANOVA
    resultados_anova = []

    # Bucle para realizar pruebas de ANOVA para cada combinación de línea y hora
    for linea in df['Linea'].unique():
        for hora in df['Horas'].unique():
            # Filtrar el DataFrame por línea y hora
            subset = df[(df['Linea'] == linea) & (df['Horas'] == hora)]

            # Crear grupos para la prueba de ANOVA
            grupos = [subset[subset['Fecha'].dt.day_name() == dia]['Qty_passangers'] for dia in df['dia_semana'].unique()]

            # Realizar la prueba de ANOVA
            resultado_anova = f_oneway(*grupos)

            # Almacenar los resultados en la lista
            resultados_anova.append({
                'Linea': linea,
                'Hora': hora,
                'Estadistica': resultado_anova.statistic,
                'Valor_p': resultado_anova.pvalue
            })

    # Convertir la lista de resultados en un DataFrame
    df_resultados = pd.DataFrame(resultados_anova)

    return df_resultados[df_resultados['Valor_p'] > 0.05]

# Uso de la función
resultado_anova = realizar_anova(df)
resultado_anova.sort_values(by=['Valor_p'], ascending=True)

,Linea,Hora,Estadistica,Valor_p
94,J,09,1.531691,0.164588
101,J,16,1.406145,0.209184
130,M,09,0.870253,0.516175


# KRUSKAL

In [7]:
import pandas as pd
from scipy.stats import kruskal

def realizar_kruskal_wallis(archivo_csv):
    """
    Realiza un test de Kruskal-Wallis para determinar si hay diferencias significativas
    en la cantidad de pasajeros entre los días de la semana para cada combinación única de línea y hora.

    Parámetros:
    - archivo_csv (str): Ruta al archivo CSV que contiene los datos.

    Retorna:
    - df_resultados (pd.DataFrame): DataFrame que contiene los resultados del test de Kruskal-Wallis.

    Explicación:
    Un valor p menor a 0.05 indica que hay evidencia suficiente para rechazar la hipótesis nula.
    La hipótesis nula (H0) asume que no hay diferencias significativas en la distribución de la cantidad
    de pasajeros entre los días de la semana para una línea y hora específicas.

    Por lo tanto:
    - Si Valor_p < 0.05: Se rechaza H0, lo que sugiere que hay diferencias significativas.
    - Si Valor_p >= 0.05: No hay evidencia suficiente para rechazar H0.
    """
    # Manipulación de fechas
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['dia_semana'] = df['Fecha'].dt.day_name()

    # Inicialización de la lista de resultados de Kruskal-Wallis
    resultados_kruskal_wallis = []

    # Bucle para realizar pruebas de Kruskal-Wallis para cada combinación de línea y hora
    for linea in df['Linea'].unique():
        for hora in df['Horas'].unique():
            # Filtrar el DataFrame por línea y hora
            subset = df[(df['Linea'] == linea) & (df['Horas'] == hora)]

            # Crear grupos para la prueba de Kruskal-Wallis
            grupos = [subset[subset['Fecha'].dt.day_name() == dia]['Qty_passangers'] for dia in df['dia_semana'].unique()]

            # Realizar la prueba de Kruskal-Wallis
            resultado_kruskal_wallis = kruskal(*grupos)

            # Almacenar los resultados en la lista
            resultados_kruskal_wallis.append({
                'Linea': linea,
                'Hora': hora,
                'Estadistica': resultado_kruskal_wallis.statistic,
                'Valor_p': resultado_kruskal_wallis.pvalue
            })

    # Convertir la lista de resultados en un DataFrame
    df_resultados = pd.DataFrame(resultados_kruskal_wallis)

    return df_resultados[df_resultados['Valor_p'] > 0.05]

# Uso de la función
resultado_kruskal_wallis = realizar_kruskal_wallis(df)
resultado_kruskal_wallis.sort_values(by=['Valor_p'], ascending=True)


,Linea,Hora,Estadistica,Valor_p


## Test de anova Por Día de la Semana: Analizar si la afluencia varía significativamente entre los diferentes días de la semana.

Por Hora del Día: Investigar si hay diferencias significativas en la cantidad de pasajeros en diferentes horas del día.

Por Días Festivos vs. Días No Festivos: Comparar la afluencia en días festivos frente a días no festivos para ver si hay una diferencia significativa.

In [9]:
import pandas as pd
from scipy.stats import f_oneway

def asignar_periodo_del_dia(hora, periodos):
    """
    Asigna un período del día a una hora dada, basado en los rangos definidos en 'periodos'.
    """
    hora = int(hora)  # Convertir la hora a entero
    for periodo, (inicio, fin) in periodos.items():
        if inicio <= hora < fin:
            return periodo
    return 'Otro'  # para horas fuera de los rangos especificados

def realizar_anova_por_periodos(df, periodos):
    """
    Realiza un análisis ANOVA para cada línea dentro de cada período del día y retorna tres DataFrames separados.

    Parámetros:
    - df (DataFrame): DataFrame de Pandas.
    - periodos (dict): Diccionario que define los períodos del día.
    """
    
    # Convertir la fecha a datetime y crear la columna de período del día
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['Periodo_Dia'] = df['Horas'].apply(lambda hora: asignar_periodo_del_dia(hora, periodos))
    
    # Diccionario para almacenar los DataFrames de resultados
    resultados_por_periodo = {periodo: [] for periodo in periodos}

    # Realizar ANOVA para cada línea y período
    for periodo in periodos:
        for linea in df['Linea'].unique():
            subset = df[(df['Linea'] == linea) & (df['Periodo_Dia'] == periodo)]
            grupos = [grupo['Qty_passangers'].values for _, grupo in subset.groupby(subset['Fecha'].dt.date) if len(grupo) > 1]

            if grupos and len(grupos) > 1:
                resultado_anova = f_oneway(*grupos)
                resultados_por_periodo[periodo].append({
                    'Linea': linea,
                    'Estadistica_F': resultado_anova.statistic,
                    'Valor_p': resultado_anova.pvalue
                })

    # Convertir los resultados a DataFrames
    for periodo in resultados_por_periodo:
        resultados_por_periodo[periodo] = pd.DataFrame(resultados_por_periodo[periodo])

    return resultados_por_periodo['Mañana'], resultados_por_periodo['Tarde'], resultados_por_periodo['Noche']

# Ejemplo de uso de la función
periodos_definidos = {'Mañana': (6, 12), 'Tarde': (12, 18), 'Noche': (18, 23)}
resultados_manana, resultados_tarde, resultados_noche = realizar_anova_por_periodos(df, periodos_definidos)



In [10]:
resultados_manana_significativos = resultados_manana[resultados_manana['Valor_p'] > 0.05]
resultados_tarde_significativos = resultados_tarde[resultados_tarde['Valor_p'] > 0.05]
resultados_noche_significativos = resultados_noche[resultados_noche['Valor_p'] > 0.05]

# Mostrar los resultados
print("Resultados Mañana (p > 0.05):")
print(resultados_manana_significativos)
print("\nResultados Tarde (p > 0.05):")
print(resultados_tarde_significativos)
print("\nResultados Noche (p > 0.05):")
print(resultados_noche_significativos)

Resultados Mañana (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_F, Valor_p]
Index: []

Resultados Tarde (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_F, Valor_p]
Index: []

Resultados Noche (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_F, Valor_p]
Index: []


Resultados
Valor F: 281.21
Valor p: Prácticamente 0 (indicativo de una probabilidad extremadamente baja de que las diferencias observadas en las medias sean debido al azar).

El alto valor F y el valor p cercano a cero indican que hay diferencias significativas en la afluencia de pasajeros entre los distintos días de la semana. Esto sugiere que el día de la semana es un factor importante que influye en la cantidad de pasajeros que utilizan el metro. La significancia estadística implica que estas diferencias no son atribuibles al azar y que hay patrones específicos en la afluencia de pasajeros según el día de la semana.

# KRUSKAL

In [12]:
import pandas as pd
from scipy.stats import kruskal

def asignar_periodo_del_dia(hora, periodos):
    """
    Asigna un período del día a una hora dada, basado en los rangos definidos en 'periodos'.
    """
    hora = int(hora)  # Convertir la hora a entero
    for periodo, (inicio, fin) in periodos.items():
        if inicio <= hora < fin:
            return periodo
    return 'Otro'  # para horas fuera de los rangos especificados

def realizar_kruskal_wallis_por_periodos(df, periodos):
    """
    Realiza un test de Kruskal-Wallis para cada línea dentro de cada período del día y retorna tres DataFrames separados.

    Parámetros:
    - df (DataFrame): DataFrame de Pandas.
    - periodos (dict): Diccionario que define los períodos del día.
    """
    
    # Convertir la fecha a datetime y crear la columna de período del día
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['Periodo_Dia'] = df['Horas'].apply(lambda hora: asignar_periodo_del_dia(hora, periodos))
    
    # Diccionario para almacenar los DataFrames de resultados
    resultados_por_periodo = {periodo: [] for periodo in periodos}

    # Realizar Kruskal-Wallis para cada línea y período
    for periodo in periodos:
        for linea in df['Linea'].unique():
            subset = df[(df['Linea'] == linea) & (df['Periodo_Dia'] == periodo)]
            grupos = [grupo['Qty_passangers'].values for _, grupo in subset.groupby(subset['Fecha'].dt.date) if len(grupo) > 1]

            if grupos and len(grupos) > 1:
                resultado_kruskal_wallis = kruskal(*grupos)
                resultados_por_periodo[periodo].append({
                    'Linea': linea,
                    'Estadistica_H': resultado_kruskal_wallis.statistic,
                    'Valor_p': resultado_kruskal_wallis.pvalue
                })

    # Convertir los resultados a DataFrames
    for periodo in resultados_por_periodo:
        resultados_por_periodo[periodo] = pd.DataFrame(resultados_por_periodo[periodo])

    return resultados_por_periodo['Mañana'], resultados_por_periodo['Tarde'], resultados_por_periodo['Noche']

# Ejemplo de uso de la función
periodos_definidos = {'Mañana': (6, 12), 'Tarde': (12, 18), 'Noche': (18, 24)}
manana_df, tarde_df, noche_df = realizar_kruskal_wallis_por_periodos(df, periodos_definidos)



In [13]:
resultados_manana_significativos1 = manana_df[manana_df['Valor_p'] > 0.05]
resultados_tarde_significativos1 = tarde_df[tarde_df['Valor_p'] > 0.05]
resultados_noche_significativos1 = noche_df[noche_df['Valor_p'] > 0.05]

# Mostrar los resultados
print("Resultados Mañana (p > 0.05):")
print(resultados_manana_significativos1)
print("\nResultados Tarde (p > 0.05):")
print(resultados_tarde_significativos1)
print("\nResultados Noche (p > 0.05):")
print(resultados_noche_significativos1)

Resultados Mañana (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_H, Valor_p]
Index: []

Resultados Tarde (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_H, Valor_p]
Index: []

Resultados Noche (p > 0.05):
  Linea  Estadistica_H   Valor_p
5     J     830.006763  0.943301
